In [5]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.0 MB 8.7 MB/s eta 0:00:01
     |████████████████████████████████| 25.7 MB 38.7 MB/s eta 0:00:01
     |████████████████████████████████| 13.9 MB 25.5 MB/s eta 0:00:01
     |████████████████████████████████| 8.5 MB 35.2 MB/s eta 0:00:01
     |████████████████████████████████| 771 kB 43.2 MB/s eta 0:00:01
     |████████████████████████████████| 302 kB 44.2 MB/s eta 0:00:01
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-macosx_10_9_universal2.whl size=1584099 sha256=ea02695c220f50974e06954cba552f8db2bbbebfce6f714dc5c3832a64eb6a41
  Stored in directory: /Users/bmoon/Library/Caches/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attemp

In [6]:
import pandas as pd
import os
import preprocess.preprocess as cbf
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [7]:
data_path = os.path.abspath(os.path.join(os.getcwd(), 'data'))

df_traveller = pd.read_csv(os.path.join(data_path, 'traveller.csv'))
df_travel = pd.read_csv(os.path.join(data_path, 'travel.csv'))
df_area = pd.read_csv(os.path.join(data_path, 'area.csv'))

/var/folders/jx/jl9n9n4d0qj3w4flzwz3w0y00000gn/T/ipykernel_7665/1656092857.py:5: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_area = pd.read_csv(os.path.join(data_path, 'area.csv'))


In [8]:
df, categorical_features = cbf.cbf_preprocess(df_traveller, df_travel, df_area)

/Users/bmoon/Desktop/moon/projects/recsys/preprocess/preprocess.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_area.dropna(subset = ['LOTNO_ADDR'], inplace = True)
/Users/bmoon/Desktop/moon/projects/recsys/preprocess/preprocess.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RESIDENCE_TIME_MIN'] = df['RESIDENCE_TIME_MIN'].replace(0,60)
/Users/bmoon/Desktop/moon/projects/recsys/preprocess/preprocess.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [10]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
        
# Pool 생성
train_pool = Pool(train_df.drop(['DGSTFN'], axis=1),
                  label=train_df['DGSTFN'],
                  cat_features=categorical_features)
test_pool = Pool(test_df.drop(['DGSTFN'], axis=1),
                 label=test_df['DGSTFN'],
                 cat_features=categorical_features)

# 모델 학습
model = CatBoostRegressor(loss_function='RMSE', eval_metric='MAE', task_type='CPU', depth=6, learning_rate=0.01, n_estimators=2000)
model.fit(train_pool, eval_set=test_pool, verbose=False)

# 검증 데이터셋에서의 MSE 계산
predictions = model.predict(test_pool)
mse = mean_squared_error(test_df['DGSTFN'], predictions)

print(mse)

CatBoostError: /Users/zomb-ml-platform-msk/go-agent-21.2.0/pipelines/BuildMaster/catboost.git/catboost/libs/train_lib/trainer_env.cpp:9: Environment for task type [GPU] not found

In [9]:
model.save_model('cbf_model.cbm', format="cbm")

NameError: name 'model' is not defined